In [ ]:
import tensorflow as tf
import datetime
import Models
import AttentionModels
import Common
import VideoDataGenerator
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

## Prep

In [ ]:
# chose split
split_no = 1

### HMDB51 Video Data with 25 Frames

In [ ]:
dataset = Common.evaluate_dataset("D:/processed_datasets/hmdb51_vid25/")
split_df = Common.get_hmdb51_split("D:/datasets/hmdb51_org_splits", split_no=split_no)
hmdb51_video_dataset = dataset.merge(split_df, on="filename")
# encode y
hmdb51_video_dataset.category = preprocessing.LabelEncoder().fit_transform(hmdb51_video_dataset.category)
hmdb51_video_dataset

### HMDB51 OptFlow Data based on 25 Frames with L=1

In [ ]:
dataset = Common.evaluate_dataset("D:/processed_datasets/hmdb51_optflowl1_npz25/")
split_df = Common.get_hmdb51_split("D:/datasets/hmdb51_org_splits", split_no=split_no)
# fix file type extensions
split_df["filename"] = split_df["filename"].str.split(".", expand=True)[0] + ".npz"
hmdb51_optflow_dataset = dataset.merge(split_df, on="filename")
hmdb51_optflow_dataset

### prepare VideoDataGenerator

In [ ]:
train_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    optflow_path=hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 1].path.tolist(),
    batch_size=1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

In [ ]:
test_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 2].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 2].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    optflow_path=hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 2].path.tolist(),
    batch_size=1,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

## TwoStream-Network (BaseResnet50v2)

### Pre-Training individual Networks

#### Video Model

In [ ]:
video_train_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    batch_size=2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    shape_format="images"
)
video_test_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_video_dataset[hmdb51_video_dataset.split == 2].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 2].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    batch_size=2,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    shape_format="images"
)

In [ ]:
video_model = AttentionModels.create_CBAM_ResNet50v2(input_shape=(224, 224, 3), classes=51)
video_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
video_model.fit(video_train_gen, epochs=1, validation_data=video_test_gen)

In [ ]:
## Alternative: Use pretrained imagenet
## video_model = tf.keras.applications.ResNet50V2()

#### OptFlow Model

In [ ]:
optflow_train_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 1].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 1].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    batch_size=2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    shape_format="images"
)
optflow_test_gen = VideoDataGenerator.VideoDataGenerator(
    hmdb51_optflow_dataset[hmdb51_optflow_dataset.split == 2].path.tolist(),
    hmdb51_video_dataset[hmdb51_video_dataset.split == 2].category.tolist(),
    num_classes = 51,
    target_size=(224, 224),
    batch_size=2,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    shape_format="images"
)

In [ ]:
optflow_model = AttentionModels.create_CBAM_ResNet50v2(input_shape=(224, 224, 2), classes=51)
optflow_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
optflow_model.fit(optflow_train_gen, epochs=1, validation_data=optflow_test_gen)

### Training combined TwoStream Model

In [ ]:
twostream_resnet50v2 = Models.assemble_TwoStreamModel(video_model, optflow_model, 51, fusion="average", recreate_top=True)

In [ ]:
log_dir = "logs/fit_twostream_25_L1/resnet50v2_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

twostream_resnet50v2.fit(
    train_gen,
    epochs=1,
    validation_data=test_gen,
    callbacks=[tensorboard_callback]
)

In [ ]:
twostream_resnet50v2.save("models/twostream_25_L1/ResNet50v2")